<a href="https://colab.research.google.com/github/DanielMerkdo/analitica_marketing/blob/main/c_modelado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#CONEXIÓN Y LECTURA BD

## Conexión BD

In [3]:
# Montar drive y crear path para sys y para directorio de trabajo

from google.colab import drive
drive.mount('/content/drive')

import sys
import os

# Define el path al directorio de trabajo actual
path = "/content/drive/My Drive/cod/analitica_salud"


sys.path.append(path) ### para importar archivo de funciones propias a través de import

os.chdir(path) ### para que por defecto suba y descargue archivos partiendo de esa ruta

Mounted at /content/drive


## Cargar Librerías

In [4]:
!pip install --quiet optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.6/386.6 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.5/242.5 kB 13.9 MB/s eta 0:00:00


In [5]:
!pip install onnx skl2onnx onnxruntime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 94.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.5/315.5 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 90.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 5.3 MB/s eta 0:00:00


In [6]:
import pandas as pd
import numpy as np
import graphviz
import optuna
import joblib
import seaborn as sns
import matplotlib.pyplot as plt
import time  # Para medir tiempos de inferencia
import onnx  # Para guardar y manejar el modelo en formato ONNX
import onnxruntime as ort  # Para ejecutar el modelo ONNX (se instala con: pip install onnxruntime)

from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.tree import export_graphviz
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,confusion_matrix, ConfusionMatrixDisplay, f1_score
from sklearn.linear_model import LogisticRegression
from skl2onnx.common.data_types import FloatTensorType

# Importar librerías para conversión a ONNX
from skl2onnx import to_onnx



##Acceso Notebook

In [ ]:
salud_model = pd.read_csv('/content/salud_modelado_final.csv')
salud_model.head()

# Modelo

## Regresión Logística

In [ ]:
x = salud_model.drop("cardio", axis=1)

y = salud_model["cardio"]

x_train,x_test,y_train,y_test = train_test_split(x,y,shuffle = True,test_size=0.30, random_state=123)


model = LogisticRegression() # definir el modelo
model.fit(x_train,y_train) # entrenar el modelo

y_pred_train = model.predict(x_train) # guardar la predicción para train
y_pred_test = model.predict(x_test) # guardar la predicción para test


y_pred_test # predicción de test
y_test # realidad de test

### Metricas de Desempeño

In [ ]:
# metricas de entrenamiento
cm1 = confusion_matrix(y_train, y_pred_train, labels=model.classes_) # guardar las clases para la matriz de confusión
disp = ConfusionMatrixDisplay(confusion_matrix=cm1,display_labels=model.classes_)
disp.plot();
print(cm1)

In [ ]:
TP=cm1[1,1] # el modelo dijo que eran 1 y en la realidad son 1, verdaderos positivos - 446
FP=cm1[0,1] # el modelo dijo que eran 1, en la realidad son 0, falsos positivos - 49
FN=cm1[1,0] # el modelo dijo que eran 0, en la realidad son 1, falsos negativos - 63
TN=cm1[0,0] # el modelo dijo que eran 0 y en la realidad son 0, verdaderos negativos - 723

print(f"Accuracy train: {accuracy_score(y_train, y_pred_train)}")
print(f'Precicion: {TP/(TP+FP)}')
print(f'Recall (Sensibilidad)): {TP/(TP+FN)}')
print(f'F1-score:', f1_score(y_train, y_pred_train, average='binary'))
print(f'Especificidad: {TN/(FP+TN)}')

In [ ]:
# Matriz de confusión:
cm = confusion_matrix(y_test, y_pred_test, labels=model.classes_) # guardar las clases para la matriz de confusión
disp = ConfusionMatrixDisplay(confusion_matrix=cm,display_labels=model.classes_)
disp.plot();
print(cm)

In [ ]:
#metricas de testeo
TP=cm[1,1] # el modelo dijo que eran 1 y en la realidad son 1, verdaderos positivos - 446
FP=cm[0,1] # el modelo dijo que eran 1, en la realidad son 0, falsos positivos - 49
FN=cm[1,0] # el modelo dijo que eran 0, en la realidad son 1, falsos negativos - 63
TN=cm[0,0] # el modelo dijo que eran 0 y en la realidad son 0, verdaderos negativos - 723

print(f"Accuracy test: {accuracy_score(y_test, y_pred_test)}")
print(f'Precicion: {TP/(TP+FP)}')
print(f'Recall (Sensibilidad)): {TP/(TP+FN)}')
print(f'F1-score:', f1_score(y_test, y_pred_test, average='binary'))
print(f'Especificidad: {TN/(FP+TN)}')

## Árbol de Decisión

In [ ]:
salud_arbol=salud_model.copy()
salud_arbol.head()

In [ ]:

# dividir en independientes y dependiente
X, y = salud_arbol.drop('cardio', axis = 1), salud_arbol['cardio']

# definir modelo
clf = tree.DecisionTreeClassifier(max_depth=10, random_state=123) #, class_weight='balanced', #
# dividir en train y test
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 123, test_size= 0.3)

# entrenar modelo
clf = clf.fit(X_train, y_train)

In [ ]:
# Convertir los valores únicos de y a cadenas
class_names = [str(cls) for cls in y.unique()]

# Generar el gráfico

dot_data = tree.export_graphviz(clf,
                                feature_names=X.columns,  # Cambiado a X.columns
                                class_names=class_names,  # Cambiado a la lista de clases
                                filled=True, rounded=True,
                                special_characters=True)

graph = graphviz.Source(dot_data)
graph

### Métricas de Desempeño

In [ ]:
# hacer la predicción sobre train
clf = clf.fit(X_train, y_train)
y_true = clf.predict(X_train)
print(metrics.classification_report(y_train, y_true, digits = 3))

In [ ]:
# hacer la predicción sobre test
clf = clf.fit(X_train, y_train)
y_true = clf.predict(X_test)
print(metrics.classification_report(y_test, y_true, digits = 3))

## Bosque Aleatorio

In [ ]:
salud_forest=salud_model.copy()
salud_forest

In [ ]:
X, y = salud_forest.drop('cardio', axis = 1), salud_forest['cardio']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 123, test_size= 0.3)

In [ ]:
# Crear el modelo
model = RandomForestClassifier(n_estimators=150,  # número de árboles
                               criterion='gini',
                               max_depth=10,  # estrategia de parada
                               min_samples_leaf=5,
                               max_features=None,
                               oob_score=False,
                               n_jobs=-1,
                               random_state=123)

# Entrenar el modelo
model.fit(X_train, y_train)


### Métricas de Desempeño

In [ ]:
# Calcular y mostrar las métricas en el conjunto de entrenamiento
print(metrics.classification_report(y_train, model.predict(X_train)))

In [ ]:
# Calcular y mostrar las métricas en el conjunto de prueba
print(metrics.classification_report(y_test, model.predict(X_test)))

# Optimización de Hiperparámetros

Se selecciona el modelo de bosque aleatorio para optimización de hiperparámetros

In [ ]:
def objective(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 50, 200),
        "max_depth": trial.suggest_int("max_depth", 5, 20),
        "max_features": trial.suggest_categorical("max_features", ["sqrt", "log2", None]),
        "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 5)
    }

    model = RandomForestClassifier(**params, random_state=123)
    score = cross_val_score(model, X_train, y_train, cv=3, scoring="f1").mean()
    return score

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=10)


In [ ]:

#Resultados
print(f"Best trial:")
print(f"  Value (Best F1): {study.best_value:.4f}")
print(f"  Params: {study.best_params}")


In [ ]:
#Obtener los mejores parámetros
best_params = study.best_params

#Crear el modelo con esos parámetros
best_model = RandomForestClassifier(**best_params, random_state=123)

#Entrenarlo
best_model.fit(X_train, y_train)

In [ ]:
y_pred_test = best_model.predict(X_test)

from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred_test))

In [ ]:
print(metrics.classification_report(y_train, model.predict(X_train)))

In [ ]:
print(metrics.classification_report(y_test, model.predict(X_test)))

In [ ]:
joblib.dump(best_model, "salidas/best_rf_optuna.pkl")

# RED NEURONAL

In [49]:
!pip install --upgrade scikit-learn
!pip install --upgrade scikit-learn
!pip install scikeras
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from sklearn.model_selection import cross_val_score
import numpy as np

In [10]:
import pandas as pd
df = pd.read_csv('data/cardio_train.csv', sep=';')
df.head()

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,2,18857,1,165,64.0,130,70,3,1,0,0,0,1
3,3,17623,2,169,82.0,150,100,1,1,0,0,1,1
4,4,17474,1,156,56.0,100,60,1,1,0,0,0,0


In [13]:
salud_cnn=df.copy()
salud_cnn.head()

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,2,18857,1,165,64.0,130,70,3,1,0,0,0,1
3,3,17623,2,169,82.0,150,100,1,1,0,0,1,1
4,4,17474,1,156,56.0,100,60,1,1,0,0,0,0


In [15]:
X = salud_cnn.iloc[:, :-1].values  # Todas las columnas excepto la última
y = salud_cnn.iloc[:, -1].values   # La última columna

In [16]:
# Dividir en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [19]:
# Escalar los datos numéricos
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [20]:
# Reformatear los datos para la CNN (asumo que cada fila es una serie de tiempo con múltiples características)
time_steps = X_train.shape[1]  # Número de características
num_features = 1
X_train = X_train.reshape(X_train.shape[0], time_steps, num_features)
X_test = X_test.reshape(X_test.shape[0], time_steps, num_features)

In [21]:
# 2. Creación del modelo CNN
model = keras.Sequential([
    layers.Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(time_steps, num_features)),
    layers.MaxPooling1D(pool_size=2),
    layers.Conv1D(filters=64, kernel_size=3, activation='relu'),
    layers.MaxPooling1D(pool_size=2),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')  # Para clasificación binaria
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [22]:
# 3. Compilación del modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [23]:
# 4. Entrenamiento del modelo
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/10
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 11s 5ms/step - accuracy: 0.6560 - loss: 0.6232 - val_accuracy: 0.7229 - val_loss: 0.5601
Epoch 2/10
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 10s 6ms/step - accuracy: 0.7242 - loss: 0.5609 - val_accuracy: 0.7263 - val_loss: 0.5558
Epoch 3/10
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.7288 - loss: 0.5544 - val_accuracy: 0.7350 - val_loss: 0.5508
Epoch 4/10
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 9s 5ms/step - accuracy: 0.7266 - loss: 0.5547 - val_accuracy: 0.7273 - val_loss: 0.5553
Epoch 5/10
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 11s 5ms/step - accuracy: 0.7289 - loss: 0.5546 - val_accuracy: 0.7309 - val_loss: 0.5542
Epoch 6/10
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.7276 - loss: 0.5547 - val_accuracy: 0.7350 - val_loss: 0.5458
Epoch 7/10
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.7303 - loss: 0.5488 - val_accuracy: 0.7339 - val_loss: 0.5477
Epoch 8/10
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 12s 5ms/step - accuracy: 0.7294 - loss: 0.55

In [24]:
# 5. Evaluación del modelo
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Accuracy: {accuracy}')

438/438 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7403 - loss: 0.5390
Accuracy: 0.7347142696380615


## Optimización de hiperparámetros

In [72]:
# Define una función para crear el modelo (esto es necesario para KerasClassifier)
def create_model(filters=32, kernel_size=3, dropout_rate=0.5):
    model = Sequential([
        Conv1D(filters=filters, kernel_size=kernel_size, activation='relu', input_shape=(time_steps, num_features), padding='same'),
        MaxPooling1D(pool_size=2),
        Conv1D(filters=filters*2, kernel_size=kernel_size, activation='relu', padding='same'),
        MaxPooling1D(pool_size=2),
        Flatten(),
        Dense(64, activation='relu'),
        Dropout(dropout_rate),
        Dense(1, activation='sigmoid')  # Para clasificación binaria
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [73]:
#  Crea un KerasClassifier
model = KerasClassifier(build_fn=create_model, verbose=0) # verbose=0 to avoid printing during GridSearchCV

# 4. Define la grilla de hiperparámetros a explorar
param_grid = {
    'filters': [32, 64],               # Número de filtros en las capas convolucionales
    'kernel_size': [3, 5],           # Tamaño del kernel en las capas convolucionales
    'dropout_rate': [0.3, 0.5],        # Tasa de dropout
    'epochs': [10, 50],                # Número de épocas
    'batch_size': [32, 64]            # Tamaño del lote
}

In [74]:
# 4. Define la grilla de hiperparámetros a explorar
param_grid = {
    'filters': [32, 64],               # Número de filtros en las capas convolucionales
    'kernel_size': [3, 5],           # Tamaño del kernel en las capas convolucionales
    'dropout_rate': [0.3, 0.5],        # Tasa de dropout
    'epochs': [10, 15],                # Número de épocas
    'batch_size': [32, 64]            # Tamaño del lote
}

In [75]:
# 5. Define el espacio de búsqueda de hiperparámetros con Hyperopt
space = {
    'filters': hp.choice('filters', [32, 64]),
    'kernel_size': hp.choice('kernel_size', [3, 5]),
    'dropout_rate': hp.uniform('dropout_rate', 0.3, 0.5), # hp.uniform for continuous values
    'epochs': hp.choice('epochs', [10, 50]),
    'batch_size': hp.choice('batch_size', [32, 64])
}

In [76]:
# 6. Define la función objetivo para Hyperopt (lo que queremos minimizar)
def objective(params):
    # Unpack the parameters
    filters = params['filters']
    kernel_size = params['kernel_size']
    dropout_rate = params['dropout_rate']
    epochs = params['epochs']
    batch_size = params['batch_size']

    # Create the model with the given hyperparameters
    model = create_model(filters=filters, kernel_size=kernel_size, dropout_rate=dropout_rate)

    # Perform cross-validation using the training data
    scores = cross_val_score(KerasClassifier(build_fn=lambda: model, epochs=epochs, batch_size=batch_size, verbose=0),
                             X_train, y_train, cv=3, scoring='accuracy')

    # Return the mean cross-validation accuracy as the loss (we want to *minimize* the *negative* accuracy)
    return {'loss': -np.mean(scores), 'status': STATUS_OK}


In [61]:
# 7. Optimización bayesiana con Hyperopt
trials = Trials()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=5,
            trials=trials)

print("Mejores hiperparámetros: ", best)


  0%|          | 0/5 [00:00<?, ?trial/s, best loss=?]

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regular

 20%|██        | 1/5 [02:15<09:01, 135.38s/trial, best loss: -0.7316964666048685]

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regular

 40%|████      | 2/5 [08:41<14:08, 282.67s/trial, best loss: -0.7316964666048685]

...

 60%|██████    | 3/5 [13:43<09:43, 291.80s/trial, best loss: -0.7316964666048685]

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regular

 80%|████████  | 4/5 [16:09<03:54, 234.18s/trial, best loss: -0.7316964666048685]

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regular

100%|██████████| 5/5 [19:37<00:00, 235.44s/trial, best loss: -0.7321250041325887]
Mejores hiperparámetros:  {'batch_size': np.int64(1), 'dropout_rate': np.float64(0.4685501891739018), 'epochs': np.int64(1), 'filters': np.int64(0), 'kernel_size': np.int64(1)}


In [69]:
# 8. Imprime los mejores hiperparámetros encontrados
print("Mejores hiperparámetros: ", best)

# Translate choice indices to actual values
best_params = {
    'filters': [32, 64][best['filters']],
    'kernel_size': [3, 5][best['kernel_size']],
    'dropout_rate': best['dropout_rate'],
    'epochs': [10, 50][best['epochs']],
    'batch_size': [32, 64][best['batch_size']]
}
print("Mejores hiperparámetros (valores): ", best_params)


Mejores hiperparámetros:  {'batch_size': np.int64(1), 'dropout_rate': np.float64(0.4685501891739018), 'epochs': np.int64(1), 'filters': np.int64(0), 'kernel_size': np.int64(1)}
Mejores hiperparámetros (valores):  {'filters': 32, 'kernel_size': 5, 'dropout_rate': np.float64(0.4685501891739018), 'epochs': 50, 'batch_size': 64}


In [70]:
# 9. Entrena el modelo con los mejores hiperparámetros en todo el conjunto de entrenamiento
best_model = create_model(filters=best_params['filters'], kernel_size=best_params['kernel_size'], dropout_rate=best_params['dropout_rate'])
best_model.fit(X_train, y_train, epochs=best_params['epochs'], batch_size=best_params['batch_size'], verbose=1)

Epoch 1/50


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


875/875 ━━━━━━━━━━━━━━━━━━━━ 7s 6ms/step - accuracy: 0.6390 - loss: 0.6355
Epoch 2/50
875/875 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - accuracy: 0.7126 - loss: 0.5760
Epoch 3/50
875/875 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - accuracy: 0.7235 - loss: 0.5624
Epoch 4/50
875/875 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.7274 - loss: 0.5553
Epoch 5/50
875/875 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/step - accuracy: 0.7311 - loss: 0.5510
Epoch 6/50
875/875 ━━━━━━━━━━━━━━━━━━━━ 9s 5ms/step - accuracy: 0.7349 - loss: 0.5442
Epoch 7/50
875/875 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - accuracy: 0.7307 - loss: 0.5486
Epoch 8/50
875/875 ━━━━━━━━━━━━━━━━━━━━ 10s 6ms/step - accuracy: 0.7311 - loss: 0.5487
Epoch 9/50
875/875 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - accuracy: 0.7318 - loss: 0.5508
Epoch 10/50
875/875 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/step - accuracy: 0.7330 - loss: 0.5469
Epoch 11/50
875/875 ━━━━━━━━━━━━━━━━━━━━ 9s 5ms/step - accuracy: 0.7303 - loss: 0.5499
Epoch 12/50
875/875 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/step - accura

In [71]:
# 10. Evalúa el modelo con los mejores hiperparámetros en el conjunto de prueba
loss, accuracy = best_model.evaluate(X_test, y_test)
print(f'Precisión en el conjunto de prueba con los mejores hiperparámetros: {accuracy}')

438/438 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.7397 - loss: 0.5431
Precisión en el conjunto de prueba con los mejores hiperparámetros: 0.7327142953872681
